# Batch Inferences

This Jupyter Notebook shows how the `nff` package interfaces with the Atomistic Simulation Environment (ASE). We assume the user went through tutorial `01_training`, so we can load the pretrained models without having to train them again.

As before, importing the dependencies:

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from ase import Atoms
from ase.md.verlet import VelocityVerlet

import nff

from nff.md.nve import Dynamics
from nff.data import Dataset
from nff.io.ase import NeuralFF, AtomsBatch
from nff.train import load_model, evaluate
import nff.utils.constants as const
from ase import units

## Loading the relevant data

We reload the dataset and create a `GraphLoader` as we did last time:

In [2]:
dataset = Dataset.from_file('data/dataset.pth.tar')

### Creating Atoms

As before, we can create an `Atoms` object from any element of the dataset. Let's take the first one, for simplicity:

In [3]:
props = dataset[0].copy()
atoms = AtomsBatch(positions=props['nxyz'][:, 1:], numbers=props['nxyz'][:, 0], props=props)

# Load trained models

Here we load two models, the idea for ensembled approach would be to evaluate the same batch twice with different models to bound the uncertainity for inference purpose

In [15]:
nff_ase_1 = NeuralFF.from_file('models/cco_1/', device=1)
nff_ase_2 = NeuralFF.from_file('models/cco_2/', device=1)

get batches 

In [16]:
from nff.utils.cuda import batch_to

batch = batch_to(atoms.get_batch(), 1)

force computed from model 1

In [18]:
nff_ase_1.model(batch)['energy_grad']

tensor([[  2.6625, -11.7257,  -1.6760],
        [-10.9157,  14.8093,   8.4685],
        [-14.1403,   0.8944,   6.1563],
        [  0.0431,   2.2324,   4.9156],
        [  1.2007,   0.6308,  -4.7388],
        [  6.2475,  -5.7065,   6.2277],
        [  8.9826,  -8.6119,  -3.2076],
        [ -1.0577,   4.6916, -10.8629],
        [  6.9773,   2.7857,  -5.2829]], device='cuda:1',
       grad_fn=<AddBackward0>)

In [19]:
nff_ase_2.model(batch)['energy_grad']

tensor([[  4.8635, -11.6434,  -1.4136],
        [-15.7632,  14.9333,   8.9940],
        [-15.3593,   1.0160,   6.6080],
        [  0.3149,   2.3005,   4.9099],
        [  0.6989,   0.7217,  -4.4088],
        [  6.7211,  -6.1006,   7.3934],
        [  9.8623, -10.2740,  -4.3793],
        [ -0.6546,   5.9508, -11.6523],
        [  9.3164,   3.0958,  -6.0514]], device='cuda:1',
       grad_fn=<AddBackward0>)

# And we want to take the average of the output

# Goal: how to evaluate on the same data with several models?\